In [4]:
# -*- coding: utf-8 -*-
# Python 3.9+
# pip install requests==2.32.3 python-dotenv==1.0.1

import os, time, hmac, hashlib, urllib.parse, requests, math, threading, random
from dataclasses import dataclass
from typing import Optional, Tuple
from decimal import Decimal, ROUND_FLOOR, ROUND_CEILING
from dotenv import load_dotenv

# ================== 設定 ==================
SYMBOL        = "BTCUSDT"
NOTIONAL_USD  = 3500.0          # 建玉の想定USDサイズ
QTY_STEP      = 0.001            # ASTERの数量刻み（整数枚想定なら 1）
SL_PCT        = 0.015           # 2% 逆行で損切り
KLINE_LIMIT   = 40             # 余裕を持って取得（この中から直近30本を使用）
CYCLE_MIN     = 20             # 20分ごとにシグナル生成
PNL_LOG_EVERY = 60             # 1分ごとに含み損益ログ
USER_AGENT    = "ThresholdRandBot/1.0"
RECV_WINDOW   = 50000

# Asterの精度（必要に応じて調整）
ASTER_PRECISION = {
    "price_tick": Decimal("0.1"),
    "price_precision": 1,
    "qty_step": Decimal(str(QTY_STEP)),
    "qty_min": Decimal(str(QTY_STEP)),
    "qty_precision": 3,   # 整数枚
}

ASTER_HOST    = "https://fapi.asterdex.com"
ASTER_KEY_HDR = "X-MBX-APIKEY"

# ================== 共通ユーティリティ ==================
def _now_ms() -> int:
    return int(time.time() * 1000)

def _hmac_sha256(secret: str, msg: str) -> str:
    return hmac.new(secret.encode("utf-8"), msg.encode("utf-8"), hashlib.sha256).hexdigest()

def _round_to_step(val: Decimal, step: Decimal, direction: str) -> Decimal:
    units = (val / step).to_integral_value(rounding=(ROUND_CEILING if direction == "up" else ROUND_FLOOR))
    return units * step

def _limit_decimals(val: Decimal, decimals: int) -> Decimal:
    if decimals <= 0:
        return Decimal(int(val))
    q = Decimal(1).scaleb(-decimals)
    return val.quantize(q, rounding=ROUND_FLOOR)

def round_down_qty_float(qty: float, step: float) -> float:
    q = Decimal(str(qty))
    s = Decimal(str(step))
    units = (q / s).to_integral_value(rounding=ROUND_FLOOR)
    return float(units * s)

def sleep_until_next_second(sec_interval: int):
    """次の interval 秒境界までスリープ（粗い同期用）"""
    now = time.time()
    next_t = math.floor(now / sec_interval) * sec_interval + sec_interval
    time.sleep(max(0.0, next_t - now))

# ================== Aster v2 HMAC クライアント ==================
@dataclass
class AsterClient:
    api_key: str
    api_secret: str
    host: str = ASTER_HOST
    price_tick: Decimal = ASTER_PRECISION["price_tick"]
    price_precision: int = ASTER_PRECISION["price_precision"]
    qty_step: Decimal = ASTER_PRECISION["qty_step"]
    qty_min: Decimal = ASTER_PRECISION["qty_min"]
    qty_precision: int = ASTER_PRECISION["qty_precision"]

    def _request(self, method: str, path: str, params: dict):
        url = self.host + path
        q = dict(params or {})
        q.setdefault("recvWindow", RECV_WINDOW)
        q["timestamp"] = _now_ms()
        qs = urllib.parse.urlencode(q, doseq=True)
        sig = _hmac_sha256(self.api_secret, qs)
        headers = {ASTER_KEY_HDR: self.api_key, "User-Agent": USER_AGENT}

        if method.upper() == "GET":
            full_url = f"{url}?{qs}&signature={sig}"
            r = requests.get(full_url, headers=headers, timeout=30)
        elif method.upper() == "POST":
            headers["Content-Type"] = "application/x-www-form-urlencoded"
            body = f"{qs}&signature={sig}"
            r = requests.post(url, data=body, headers=headers, timeout=30)
        elif method.upper() == "DELETE":
            headers["Content-Type"] = "application/x-www-form-urlencoded"
            body = f"{qs}&signature={sig}"
            r = requests.delete(url, data=body, headers=headers, timeout=30)
        else:
            raise ValueError("Unsupported method")

        if r.status_code >= 400:
            raise RuntimeError(f"Aster {method} {path} failed: {r.status_code} {r.text[:500]}")
        try:
            return r.json()
        except Exception:
            return r.text

    # precision helpers
    def fmt_qty(self, qty: float, direction: str = "down") -> str:
        v = Decimal(str(qty))
        v = _round_to_step(v, self.qty_step, direction)
        if v < max(self.qty_min, self.qty_step):
            v = self.qty_step
        v = _limit_decimals(v, self.qty_precision)
        fmt = f"{{0:.{self.qty_precision}f}}"
        return fmt.format(v) if self.qty_precision > 0 else str(int(v))

    def fmt_price(self, price: float, direction: str) -> str:
        v = Decimal(str(price))
        v = _round_to_step(v, self.price_tick, direction)
        v = _limit_decimals(v, self.price_precision)
        return f"{v:.{self.price_precision}f}".rstrip("0").rstrip(".")

    # public APIs
    def get_mark_price(self, symbol: str) -> float:
        d = self._request("GET", "/fapi/v1/premiumIndex", {"symbol": symbol})
        item = d[0] if isinstance(d, list) else d
        return float(item["markPrice"])

    def get_position(self, symbol: str) -> Tuple[float, float]:
        d = self._request("GET", "/fapi/v2/positionRisk", {"symbol": symbol})
        item = d[0] if isinstance(d, list) else d
        qty = float(item.get("positionAmt", 0.0))   # +long / -short
        ep  = float(item.get("entryPrice", 0.0))
        return qty, ep

    def place_market(self, symbol: str, side: str, qty: float, reduce_only: bool = False):
        q_str = self.fmt_qty(qty, direction="down")
        p = {
            "symbol": symbol,
            "side": side,  # "BUY"/"SELL"
            "type": "MARKET",
            "quantity": q_str,
            "positionSide": "BOTH",
        }
        if reduce_only:
            p["reduceOnly"] = "true"
        return self._request("POST", "/fapi/v1/order", p)

    def place_stop_market_close(self, symbol: str, trigger_price: float, side_to_close: str, fallback_qty: float = None):
        direction = "down" if side_to_close == "SELL" else "up"
        stop_str  = self.fmt_price(trigger_price, direction=direction)

        # 優先: closePosition=true
        pA = {
            "symbol": symbol, "side": side_to_close, "type": "STOP_MARKET",
            "stopPrice": stop_str, "closePosition": "true", "positionSide": "BOTH",
        }
        try:
            return self._request("POST", "/fapi/v1/order", pA)
        except RuntimeError:
            # フォールバック: qty + reduceOnly
            if not fallback_qty or fallback_qty <= 0:
                try:
                    pos = self._request("GET", "/fapi/v2/positionRisk", {"symbol": symbol})
                    item = pos[0] if isinstance(pos, list) else pos
                    fallback_qty = abs(float(item.get("positionAmt", 0.0)))
                except Exception:
                    fallback_qty = 0.0
            if fallback_qty <= 0:
                raise
            q_str = self.fmt_qty(fallback_qty, direction="down")
            pB = {
                "symbol": symbol, "side": side_to_close, "type": "STOP_MARKET",
                "stopPrice": stop_str, "quantity": q_str, "reduceOnly": "true", "positionSide": "BOTH",
            }
            return self._request("POST", "/fapi/v1/order", pB)

    def cancel_all(self, symbol: str):
        try:
            orders = self._request("GET", "/fapi/v1/openOrders", {"symbol": symbol})
            if isinstance(orders, list):
                for o in orders:
                    oid = o.get("orderId")
                    if oid is not None:
                        self._request("DELETE", "/fapi/v1/order", {"symbol": symbol, "orderId": oid})
        except Exception as e:
            print("[Aster] cancel_all error:", e)

    def get_klines(self, symbol: str, interval: str = "1m", limit: int = 30):
        d = self._request("GET", "/fapi/v1/klines", {"symbol": symbol, "interval": interval, "limit": limit})
        rows = []
        for r in d:
            rows.append({
                "openTime": int(r[0]),
                "open": float(r[1]),
                "high": float(r[2]),
                "low":  float(r[3]),
                "close":float(r[4]),
            })
        rows.sort(key=lambda x: x["openTime"])
        return rows

# ================== BOT 本体 ==================
class ThresholdRandBot:
    def __init__(self, aster: AsterClient):
        self.aster = aster
        self._stop_flag = False
        self._last_sl_order_set = False

    # --- 数量計算 ---
    def compute_qty(self, price: float) -> float:
        raw = NOTIONAL_USD / max(1e-12, price)
        return max(round_down_qty_float(raw, float(ASTER_PRECISION["qty_step"])), float(ASTER_PRECISION["qty_min"]))

    # --- 含み損益計算 ---
    def _calc_upnl(self, entry_price: float, qty: float, last_price: float) -> Tuple[float, float]:
        if abs(qty) < 1e-12 or entry_price <= 0 or last_price <= 0:
            return 0.0, 0.0
        if qty > 0:   # LONG
            pnl = (last_price - entry_price) * qty
        else:         # SHORT
            pnl = (entry_price - last_price) * abs(qty)
        denom = entry_price * abs(qty)
        return pnl, (pnl / denom) if denom > 0 else 0.0

    # --- 30本の1分足からしきい値を作る ---
    def compute_threshold_from_klines(self) -> Tuple[float, float, int, int]:
        kl = self.aster.get_klines(SYMBOL, interval="1m", limit=KLINE_LIMIT)
        if len(kl) < 30:
            raise RuntimeError(f"not enough klines: got {len(kl)}")
        last30 = kl[-30:]
        ups = sum(1 for x in last30 if x["close"] > x["open"])
        downs = 30 - ups  # 同値はdown側に寄せる（任意。必要なら別扱い可能）
        threshold = ups / 30.0
        return threshold, last30[-1]["close"], ups, downs

    # --- エントリー（SL発注もセット） ---
    def enter_direction(self, direction: str):
        mark = self.aster.get_mark_price(SYMBOL)
        qty, ep = self.aster.get_position(SYMBOL)
        has_pos = abs(qty) > 1e-12

        # 既存ポジがあれば一旦クローズ（方向不問でドテン）
        if has_pos:
            side_close = "SELL" if qty > 0 else "BUY"
            print(f"[exit] reduce-only {side_close} qty={abs(qty)}")
            try:
                self.aster.place_market(SYMBOL, side_close, abs(qty), reduce_only=True)
                self.aster.cancel_all(SYMBOL)  # 残っているSL等を一掃
            except Exception as e:
                print("[exit error]", e)
                # 失敗しても続行はしない（次サイクルで再試行）
                return

        # 新規エントリー
        new_qty = self.compute_qty(mark)
        if direction.upper() == "LONG":
            side = "BUY"
            sl_price = mark * (1 - SL_PCT)
        else:
            side = "SELL"
            sl_price = mark * (1 + SL_PCT)

        print(f"[enter] {direction.upper()} qty={new_qty} @~{mark:.6f} (SL {sl_price:.6f})")
        try:
            self.aster.place_market(SYMBOL, side, new_qty, reduce_only=False)
            # SL（クローズ用 STOP_MARKET）
            try:
                close_side = "SELL" if side == "BUY" else "BUY"
                self.aster.place_stop_market_close(SYMBOL, sl_price, side_to_close=close_side, fallback_qty=new_qty)
                self._last_sl_order_set = True
            except Exception as e:
                print("[warn] SL setup failed:", e)
                self._last_sl_order_set = False
        except Exception as e:
            print("[enter error]", e)
            # オープン失敗時は次サイクル待ち

    # --- 1回のシグナルサイクル ---
    def run_signal_cycle_once(self):
        try:
            threshold, last_close, ups, downs = self.compute_threshold_from_klines()
            u = random.random()  # 0〜1
            direction = "LONG" if u >= threshold else "SHORT"

            print(f"[signal] ups={ups}, downs={downs}, threshold={threshold:.3f}, rand={u:.3f} → {direction}")
            self.enter_direction(direction)
        except Exception as e:
            print("[signal cycle error]", e)

    # --- 1分ごとの含み損益ログ（別スレッド） ---
    def start_pnl_logger(self):
        def loop():
            while not self._stop_flag:
                try:
                    qty, ep = self.aster.get_position(SYMBOL)
                    mark = self.aster.get_mark_price(SYMBOL)
                    pnl, pnl_pct = self._calc_upnl(ep, qty, mark)
                    side = ("LONG" if qty > 0 else ("SHORT" if qty < 0 else "FLAT"))
                    print(f"[PnL {time.strftime('%Y-%m-%d %H:%M:%S')}] side={side} qty={qty:.4f} ep={ep:.6f} mark={mark:.6f} "
                          f"uPnL=${pnl:.2f} ({pnl_pct*100:.2f}%)")
                except Exception as e:
                    print("[pnl logger error]", e)
                sleep_until_next_second(PNL_LOG_EVERY)
        t = threading.Thread(target=loop, daemon=True)
        t.start()

    # --- 20分ごとのスケジューラ（起動直後に1回実行） ---
    def start_scheduler(self):
        def loop():
            # 起動直後に1回
            self.run_signal_cycle_once()
            # 以降は20分ごと
            while not self._stop_flag:
                time.sleep(CYCLE_MIN * 60)
                self.run_signal_cycle_once()
        t = threading.Thread(target=loop, daemon=True)
        t.start()

    def stop(self):
        self._stop_flag = True

# ================== 起動 ==================
def main():
    load_dotenv("keys.env")
    aster = AsterClient(
        api_key=os.getenv("ASTER_API_KEY") or "",
        api_secret=os.getenv("ASTER_API_SECRET") or "",
    )

    print("==============================================")
    print("[START] ThresholdRandBot booting...")
    print(f" Symbol={SYMBOL}")
    print(f" Cycle={CYCLE_MIN}min, Kline=1m x30 (limit={KLINE_LIMIT})")
    print(f" NOTIONAL_USD={NOTIONAL_USD}, QTY_STEP={QTY_STEP}, SL_PCT={SL_PCT:.2%}")
    print(" - 起動直後に1回サイクル実行、その後20分ごとに実行")
    print(" - 含み損益は1分ごとにコンソールへ表示")
    print("==============================================")

    bot = ThresholdRandBot(aster)
    bot.start_pnl_logger()
    bot.start_scheduler()

    try:
        while True:
            time.sleep(3600)  # メインスレッドは生かしておく
    except KeyboardInterrupt:
        print("\n[STOP] KeyboardInterrupt")
        bot.stop()
        aster.cancel_all(SYMBOL)
        print("[DONE]")

if __name__ == "__main__":
    main()


[START] ThresholdRandBot booting...
 Symbol=BTCUSDT
 Cycle=20min, Kline=1m x30 (limit=40)
 NOTIONAL_USD=3500.0, QTY_STEP=0.001, SL_PCT=1.50%
 - 起動直後に1回サイクル実行、その後20分ごとに実行
 - 含み損益は1分ごとにコンソールへ表示
[signal] ups=16, downs=14, threshold=0.533, rand=0.734 → LONG
[PnL 2025-09-27 21:55:34] side=FLAT qty=0.0000 ep=0.000000 mark=109363.304148 uPnL=$0.00 (0.00%)
[enter] LONG qty=0.032 @~109363.303769 (SL 107722.854212)
[PnL 2025-09-27 21:56:01] side=LONG qty=0.0320 ep=109406.600000 mark=109378.045070 uPnL=$-0.91 (-0.03%)
[PnL 2025-09-27 21:57:00] side=LONG qty=0.0320 ep=109406.600000 mark=109390.488389 uPnL=$-0.52 (-0.01%)
[PnL 2025-09-27 21:58:00] side=LONG qty=0.0320 ep=109406.600000 mark=109408.462338 uPnL=$0.06 (0.00%)
[PnL 2025-09-27 21:59:01] side=LONG qty=0.0320 ep=109406.600000 mark=109382.378077 uPnL=$-0.78 (-0.02%)
[PnL 2025-09-27 22:00:01] side=LONG qty=0.0320 ep=109406.600000 mark=109393.519661 uPnL=$-0.42 (-0.01%)
[PnL 2025-09-27 22:01:01] side=LONG qty=0.0320 ep=109406.600000 mark=1094